# The Python Standard Library

## Subjective overview

## 27. Debugging and Profiling

* 27.1. bdb — Debugger framework
* 27.2. faulthandler — Dump the Python traceback
* **27.3. pdb — The Python Debugger**
* **27.4. The Python Profilers**
* **27.5. timeit — Measure execution time of small code snippets**
* 27.6. trace — Trace or track Python statement execution
* 27.7. tracemalloc — Trace memory allocations

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import argparse


def doit():
    print('Done')


def main(dry_run=False):
    doit()


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Demo script")
    parser.add_argument('-d', '--dry_run', action='store_true',
                        help='don\'t execute any actions, just log them')
    options = parser.parse_args()
    main(dry_run=options.dry_run)


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import argparse


def doit():
    import pdb; pdb.set_trace()
    print('Done')


def main(dry_run=False):
    doit()


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Demo script")
    parser.add_argument('-d', '--dry_run', action='store_true',
                        help='don\'t execute any actions, just log them')
    options = parser.parse_args()
    main(dry_run=options.dry_run)


```
% python demo.py

> /home/jwas/src/pystok-stdlib/demo.py(9)doit()
-> print('Done')
(Pdb) 
```

```
% python -m pdb demo.py 

> /home/jwas/src/pystok-stdlib/demo.py(4)<module>()
-> import argparse
(Pdb) b doit
Breakpoint 1 at /home/jwas/src/pystok-stdlib/demo.py:7
(Pdb) run -d
Restarting demo.py with arguments:
	demo.py
> /home/jwas/src/pystok-stdlib/demo.py(4)<module>()
-> import argparse
(Pdb) 
```

```
% python -m timeit '"%s" % 5'
100000000 loops, best of 3: 0.00672 usec per loop
```

```
% python -m timeit '"%s" % 5'
100000000 loops, best of 3: 0.00672 usec per loop

% python -m timeit '"{}".format(5)'
10000000 loops, best of 3: 0.115 usec per loop
```

```
% python -m timeit '"%s" % 5'
100000000 loops, best of 3: 0.00672 usec per loop

% python -m timeit '"{}".format(5)'
10000000 loops, best of 3: 0.115 usec per loop

% ipython
Python 3.6.4 (default, Dec 23 2017, 19:07:07) 
Type 'copyright', 'credits' or 'license' for more information
IPython 6.2.1 -- An enhanced Interactive Python. Type '?' for help.

In [1]: %timeit "{}".format(5)
117 ns ± 0.319 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
```

In [ ]:
import logging
import time
from functools import wraps

logger = logging.getLogger(__name__)


def timelog(method):
    @wraps(method)
    def wrapper(*args, **kwargs):
        before = time.perf_counter()
        result = method(*args, **kwargs)
        after = time.perf_counter()

        logger.debug('Timed %s (%s, %s): %.2f s',
                     method.__name__, args, kwargs, after - before)
        return result

    return wrapper

In [ ]:
@timelog
def doit():
    time.sleep(5)
    print('Done')


```
% ./demo.py 
Done
2018-01-04 22:37:50,908 DEBUG decorators Timed doit ((), {}): 5.01 s
```

In [ ]:
# https://gist.github.com/nealtodd/2489618
def proflog(sort_args=['cumulative'], print_args=[10]):
    profiler = Profile()

    def decorator(method):
        @wraps(method)
        def wrapper(*args, **kwargs):
            try:
                result = profiler.runcall(method, *args, **kwargs)
            finally:
                s = io.StringIO()
                stats = pstats.Stats(profiler, stream=s)
                stats.strip_dirs().sort_stats(*sort_args).\
                    print_stats(*print_args)
                logger.debug('Profiled %s (%s, %s): %s',
                             method.__name__, args, kwargs, s.getvalue())
            return result
        return wrapper
    return decorator

```
% ./demo.py 
Done
2018-01-04 22:45:24,738 DEBUG decorators Profiled doit ((), {}):          4 function calls in 5.005 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.005    5.005 demo.py:11(doit)
        1    5.005    5.005    5.005    5.005 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
```

## 26. Development Tools

* 26.1. typing — Support for type hints
* 26.2. pydoc — Documentation generator and online help system
* **26.3. doctest — Test interactive Python examples**
* 26.4. unittest — Unit testing framework
* 26.5. unittest.mock — mock object library
* 26.6. unittest.mock — getting started
* 26.7. 2to3 - Automated Python 2 to 3 code translation
* 26.8. test — Regression tests package for Python
* 26.9. test.support — Utilities for the Python test suite
